In [2]:
import numpy as np
from mydx import mydx, fmm_pcm
#mydx.init()
#nbasis, ngrid, nsph = mydx.get_sizes()
#lmax = int(nbasis**0.5)-1
#print(nbasis, lmax, ngrid)

In [3]:
# COMPLEX
lmax = 2
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1
sph2_c = np.array([.0, 0, 0.5])
sph2_r = 1
src1 = np.array([0.0, 0.5, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2m_complex(src1-sph1_c, lmax)
coef2_m = fmm_pcm.p2m_complex(src1-sph2_c, lmax)
print(fmm_pcm.m2p_complex(dst1-sph1_c, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_complex(dst1-sph2_c, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)
coef1_m_normal = fmm_pcm.p2m_complex_normal(src1-sph1_c, lmax)
coef2_m_normal = fmm_pcm.p2m_complex_normal(src1-sph2_c, lmax)
print(fmm_pcm.m2p_complex_normal(dst1-sph1_c, lmax, coef1_m_normal), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_complex_normal(dst1-sph2_c, lmax, coef2_m_normal), np.linalg.norm(src1-dst1)**-1)

0.3127313958110473 0.31234752377721214
0.3126262395925993 0.31234752377721214
0.3127313958110473 0.31234752377721214
0.3126262395925993 0.31234752377721214


In [4]:
# REAL
lmax = 17
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1
sph2_c = np.array([1.0, 1, 1])
sph2_r = 1
src1 = np.array([0.0, 0.8, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2m_real(src1-sph1_c, lmax)
coef2_m = fmm_pcm.p2m_real(src1-sph2_c, lmax)
print(fmm_pcm.m2p_real(dst1-sph1_c, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_real(dst1-sph2_c, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)
coef1_m_normal = fmm_pcm.p2m_real_normal(src1-sph1_c, lmax)
coef2_m_normal = fmm_pcm.p2m_real_normal(src1-sph2_c, lmax)
print(fmm_pcm.m2p_real_normal(dst1-sph1_c, lmax, coef1_m_normal), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_real_normal(dst1-sph2_c, lmax, coef2_m_normal), np.linalg.norm(src1-dst1)**-1)
#coef2_m_m2m = fmm_pcm.m2m_complex_translate_z(sph1_c[2]-sph2_c[2], lmax, coef1_m)
#print(coef1_m)
#print(coef2_m)
#print(coef2_m_m2m)
#print(fmm_pcm.m2p_complex_normal(dst1-sph2_c, lmax, coef2_m_m2m), np.linalg.norm(src1-dst1)**-1)

0.3254722774521031 0.32547227745205964
0.32525817347924185 0.32547227745205964
0.3254722774521031 0.32547227745205964
0.3252581734792418 0.32547227745205964


In [5]:
# Normalized real spherical harmonics
lmax = 60
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1.5
sph2_c = np.array([1.0, 1, 1])
sph2_r = 2*3**0.5 + 1.5
src1 = np.array([0.0, 0.8, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2o(src1-sph1_c, sph1_r, lmax)
coef2_m = fmm_pcm.p2o(src1-sph2_c, sph2_r, lmax)
print(fmm_pcm.o2p(dst1-sph1_c, sph1_r, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.o2p(dst1-sph2_c, sph2_r, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)

0.3254722774520597 0.32547227745205964
0.3254718954621216 0.32547227745205964


In [6]:
# Compute M2M matrix by using P2M and M2P
lmax = 20
nparticles = 4000
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 2
sph2_c = np.array([0, 0, 2], dtype=np.float64)
sph2_r = 4
particles = np.zeros((nparticles, 3), dtype=np.float64)
mat1 = np.zeros((nparticles, (lmax+1)**2), dtype=np.float64)
mat2 = np.zeros((nparticles, (lmax+1)**2), dtype=np.float64)
np.random.seed(0)
for i in range(nparticles):
    tmp = np.random.randn(3)
    tmp = tmp / np.linalg.norm(tmp)
    particles[i] = sph1_c + 0.5 * sph1_r * tmp
    mat1[i] = fmm_pcm.p2o(particles[i]-sph1_c, sph1_r, lmax)
    mat2[i] = fmm_pcm.p2o(particles[i]-sph2_c, sph2_r, lmax)
q, r = np.linalg.qr(mat1)
#print(r)
m2m = np.linalg.solve(r, q.T @ mat2).T
print(np.linalg.norm(m2m @ mat1[0] - mat2[0]) / np.linalg.norm(mat2[0]))
#print(m2m)
print(np.max(m2m[1:]), np.min(m2m))
mat3 = fmm_pcm.o2o_baseline(sph1_c-sph2_c, sph1_r, sph2_r, lmax, mat1[0])
#print(m2m[10,0])
print(mat1[0][:10])
print(mat2[0][:10])
print(mat3[:10])
print(np.linalg.norm(mat3-mat2[0]) / np.linalg.norm(mat2[0]))

3.7070462957000125e-14
0.57282196186948 -0.8660254037844352
[ 0.28209479 -0.04753238  0.11625866  0.20954164 -0.04558156 -0.02528973
 -0.02527926  0.11148719  0.09530098 -0.03112826]
[ 0.28209479 -0.02376619 -0.18617192  0.10477082 -0.01139539  0.02024897
  0.07633132 -0.08926554  0.02382524 -0.00389103]
[ 0.28209479 -0.02376619 -0.18617192  0.10477082 -0.01139539  0.02024897
  0.07633132 -0.08926554  0.02382524 -0.00389103]
1.0957888172155602e-16


In [7]:
# CHECK O2O (scaled M2M)
lmax = 100
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 2
sph2_c = np.array([0, 0, 2], dtype=np.float64)
sph2_r = 4
dst1 = np.array([4.2, 0., 2.])
coef1_m = np.random.randn((lmax+1)**2)
coef2_m = fmm_pcm.o2o_baseline(sph1_c-sph2_c, sph1_r, sph2_r, lmax, coef1_m)
print(fmm_pcm.o2p(dst1-sph1_c, sph1_r, lmax, coef1_m), fmm_pcm.o2p(dst1-sph2_c, sph2_r, lmax, coef2_m))

0.5330734000100301 0.5331000096319007


In [8]:
# Check O2O matrix
lmax = 9
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 2
sph2_c = np.array([0, 0, 2], dtype=np.float64)
sph2_r = 4
o2o_mat = fmm_pcm.o2o_matrix(sph1_c-sph2_c, sph1_r, sph2_r, lmax)
print(o2o_mat)
print(np.linalg.svd(o2o_mat)[1])

[[ 1.          0.          0.         ...  0.          0.
   0.        ]
 [-0.          0.5         0.         ...  0.          0.
   0.        ]
 [-0.8660254   0.          0.5        ...  0.          0.
   0.        ]
 ...
 [-0.          0.         -0.         ...  0.00195312  0.
   0.        ]
 [ 0.          0.         -0.         ...  0.          0.00195312
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.00195312]]
[1.90398573e+00 1.50412708e+00 1.50412708e+00 1.33143065e+00
 1.07017646e+00 1.07017646e+00 8.78914777e-01 8.78914777e-01
 7.59872897e-01 6.83805265e-01 6.83805265e-01 5.11251903e-01
 5.11251903e-01 4.21168935e-01 4.21168935e-01 3.90498441e-01
 3.90498441e-01 3.75312156e-01 2.64191372e-01 2.64191372e-01
 2.03750940e-01 2.03750940e-01 1.97347586e-01 1.97347586e-01
 1.72008350e-01 1.72008350e-01 1.59441626e-01 1.20544485e-01
 1.20544485e-01 8.67570883e-02 8.67570883e-02 8.66111327e-02
 8.66111327e-02 6.89765481e-02 6.89765481e-02 5.99012268e-02